# Проект по SQL - Анализ базы данных сервиса книг по подписке

# Описание проекта
Компания купила крупный сервис для чтения книг по подписке. <br>
Задача аналитика — проанализировать базу данных с информацией о книгах, издательствах, авторах, а также пользовательскими обзорами книг.  <br>
Эти данные помогут сформулировать ценностное предложение для нового продукта.

# Описание данных

**Таблица `books`** <br>
Содержит данные о книгах: <br>
* book_id — идентификатор книги;
* author_id — идентификатор автора;
* title — название книги;
* num_pages — количество страниц;
* publication_date — дата публикации книги;
* publisher_id — идентификатор издателя.


**Таблица `authors`** <br>
Содержит данные об авторах:<br>
* author_id — идентификатор автора;
* author — имя автора.


**Таблица `publishers`** <br>
Содержит данные об издательствах: <br>
* publisher_id — идентификатор издательства;
* publisher — название издательства;


**Таблица `ratings`** <br>
Содержит данные о пользовательских оценках книг: <br>
* rating_id — идентификатор оценки;
* book_id — идентификатор книги;
* username — имя пользователя, оставившего оценку;
* rating — оценка книги.


**Таблица `reviews`** <br>
Содержит данные о пользовательских обзорах: <br>
* review_id — идентификатор обзора;
* book_id — идентификатор книги;
* username — имя автора обзора;
* text — текст обзора.


# Задачи в рамках проекта:
1. Посчитать, сколько книг вышло после 1 января 2000 года;
2. Для каждой книги посчитать количество обзоров и среднюю оценку;
3. Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
4. Определить автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
5. Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

## Импорт библиотек и подключение к базе данных

In [ ]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

In [10]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

In [ ]:
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [11]:
#Создаем функцию, выводящую результат SQL-запроса:
def result(query):
    return pd.io.sql.read_sql(query, con = engine)

## Запросы к таблицам books, authors, publishers, ratings, reviews

In [20]:
# Выведем первые пять строк всех пяти таблиц с помощью функции:
data = ['books', 'authors', 'publishers', 'ratings', 'reviews']
for table in data:
    print(f'\nТаблица: {table}')
    tab = '''SELECT * FROM {} LIMIT 5'''.format(table)
    display(result(tab))


Таблица: books


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268



Таблица: authors


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd



Таблица: publishers


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company



Таблица: ratings


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2



Таблица: reviews


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Задачи в рамках проекта

### Посчитать, сколько книг вышло после 1 января 2000 года

In [13]:
book_count = '''
SELECT COUNT(book_id)
FROM books
WHERE publication_date > '2000-01-01';
'''
result(book_count)

,count
0,819


**Вывод:** 819 книг вышло после 1 января 2000 года. 

### Для каждой книги посчитать количество обзоров и среднюю оценку

Для этого нам придется объединить три таблицы: books, reviews и ratings.

In [14]:
reviews_ratings = '''
SELECT
  books.title,
  COUNT (DISTINCT review_id) AS review_count,
  ROUND(AVG(rating),2) AS avg_rating
FROM
  books
LEFT JOIN
  reviews ON reviews.book_id = books.book_id
LEFT JOIN
  ratings ON ratings.book_id = books.book_id
GROUP BY
  books.title
ORDER BY
    review_count DESC,
    avg_rating DESC;  
'''
result(reviews_ratings)

,title,review_count,avg_rating
0,Memoirs of a Geisha,8,4.14
1,Twilight (Twilight #1),7,3.66
2,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.41
3,Harry Potter and the Chamber of Secrets (Harry...,6,4.29
4,The Book Thief,6,4.26
...,...,...,...
994,Disney's Beauty and the Beast (A Little Golden...,0,4.00
995,Essential Tales and Poems,0,4.00
996,Leonardo's Notebooks,0,4.00
997,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.67


In [6]:
reviews_ratings = '''
SELECT
  books.title,
  COUNT (DISTINCT review_id) AS review_count,
  ROUND(AVG(rating),2) AS avg_rating
FROM
  books
LEFT JOIN
  reviews ON reviews.book_id = books.book_id
LEFT JOIN
  ratings ON ratings.book_id = books.book_id
GROUP BY
  books.title
ORDER BY
    avg_rating DESC,
    review_count DESC;   
'''
result(reviews_ratings)

,title,review_count,avg_rating
0,A Dirty Job (Grim Reaper #1),4,5.00
1,Moneyball: The Art of Winning an Unfair Game,3,5.00
2,School's Out—Forever (Maximum Ride #2),3,5.00
3,Captivating: Unveiling the Mystery of a Woman'...,2,5.00
4,Evening Class,2,5.00
...,...,...,...
994,The World Is Flat: A Brief History of the Twen...,3,2.25
995,Drowning Ruth,3,2.00
996,Junky,2,2.00
997,His Excellency: George Washington,2,2.00


**Выводы:** <br>
* получилось 999 строк-книг с количеством отзывов от 0 до 8;
* Рейтинг представлен в диапазоне от 1.5 до 5;
* лидер по количеству отзывов - книга Memoirs of a Geisha, 8 откликов;
* лучший рейтинг у книг A Dirty Job (Grim Reaper), Moneyball: The Art of Winning an Unfair Game и т.д.;
* отзывов у книг с 5 баллами рейтинга от 2 до 4, в среднем - 3.

### Определить издательство, которое выпустило наибольшее число книг толще 50 страниц

In [21]:
top_publishers = '''
SELECT 
  publisher,
  COUNT (book_id) as total_books
FROM 
  books
LEFT JOIN
  publishers AS p ON p.publisher_id=books.publisher_id
WHERE
  num_pages>50
GROUP BY
  publisher
ORDER BY
  total_books DESC
LIMIT 3;
'''
result(top_publishers)

,publisher,total_books
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25


**Вывод:** <br>
* в топ-3 входят издательства Pengiun Books, Vintage, Grand Central Publishing;
* несомненный лидер - Penguin Books - 42 изданных книги толщиной более 50 страниц.    
    

### Определить автора с самой высокой средней оценкой книг
Учитывать только книги с 50 и более оценками.

In [24]:
credited_authors = '''     
WITH
one AS (SELECT book_id,
             COUNT(rating_id) as c_rating
             FROM ratings 
       GROUP BY 1
       HAVING COUNT(rating_id) >=50),
two AS (SELECT au.author_id,
             au.author,
             b.book_id,
             AVG(rating) AS avg_rating
       FROM authors AS au
       LEFT JOIN books AS b USING(author_id)
       LEFT JOIN ratings AS r ON b.book_id=r.book_id
       GROUP BY 1,3)
SELECT two.author,
       ROUND(AVG(two.avg_rating),3) as top_rating
       FROM one LEFT JOIN two USING(book_id)
       GROUP BY 1
       ORDER BY 2 DESC
       LIMIT 3;      
'''
result(credited_authors)

,author,top_rating
0,J.K. Rowling/Mary GrandPré,4.284
1,Markus Zusak/Cao Xuân Việt Khương,4.264
2,J.R.R. Tolkien,4.258


**Вывод:** <br>
* Лидер по рейтингу книг с 50 и более оценками - **J.K. Rowling, 4.284**. Сага о Гарри Поттере непобедима! 
* На втором месте - **Markus Zusak, 4.264**. Легендарные "Книжный вор", "Глиняный мост".
* На почетном третьм месте - **J.R.R. Tolkien, 4.258**. Ставшие уже классикой "Хоббит" и  "Властелин колец".


### Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [9]:
avg_reviews = '''     
WITH
one AS (SELECT DISTINCT(r.username) AS username,
            COUNT(DISTINCT(r.review_id)) as c_review,
            COUNT(DISTINCT(two.rating_id)) as c_ratings 
       FROM reviews AS r
       LEFT JOIN ratings AS two USING(username)  
       GROUP BY 1)
SELECT ROUND(AVG(one.c_review),0) AS avg_reviews
       FROM one 
       WHERE one.c_ratings > 48
'''
result(avg_reviews)

,avg_reviews
0,24.0


**Вывод:**
* 24 обзора - вот среднее количество обзоров от пользователей, поставивших более 48 оценок. 
* То есть на каждая вторая оценка сопровождается рецензией на книгу.

### Oбщий вывод:
Мы проанализировали базу данных с информацией о книгах, издательствах, авторах, а также пользовательскими обзорами книг для того, чтобы помочь коллегам составить ценностное предложение для нового продукта.

**Резюмируем результат:**
* 819 книг вышло после 1 января 2000 г.
* рейтинг представлен в диапазоне от 1.5 до 5.
* лучший рейтинг у книг A Dirty Job (Grim Reaper), Moneyball: The Art of Winning an Unfair Game.
* лидер по количеству отзывов - книга Memoirs of a Geisha, 8 откликов.
* отзывов у книг с 5 баллами рейтинга от 2 до 4, в среднем - 3.
* в топ-3 издательств входят Pengiun Books, Vintage, Grand Central Publishing.
* несомненный лидер - Penguin Books - 42 изданных книги толщиной более 50 страниц.
* Лидер по рейтингу книг с 50 и более оценками - J.K. Rowling, 4.284.
* На втором месте рейтинга книг - Markus Zusak, 4.264. Легендарные "Книжный вор", "Глиняный мост".
* На почетном третьм месте рейтинга книг - J.R.R. Tolkien, 4.258. Ставшие уже классикой "Хоббит" и "Властелин колец".
* 24 обзора  - вот среднее количество обзоров от пользователей, поставивших более 48 оценок.
* То есть на каждая вторая оценка сопровождается рецензией на книгу.